In [5]:
import numpy as np
import pandas as pd

In [6]:
np.random.seed(1)
X=np.random.randn(4,10)
print(X)

[[ 1.62434536 -0.61175641 -0.52817175 -1.07296862  0.86540763 -2.3015387
   1.74481176 -0.7612069   0.3190391  -0.24937038]
 [ 1.46210794 -2.06014071 -0.3224172  -0.38405435  1.13376944 -1.09989127
  -0.17242821 -0.87785842  0.04221375  0.58281521]
 [-1.10061918  1.14472371  0.90159072  0.50249434  0.90085595 -0.68372786
  -0.12289023 -0.93576943 -0.26788808  0.53035547]
 [-0.69166075 -0.39675353 -0.6871727  -0.84520564 -0.67124613 -0.0126646
  -1.11731035  0.2344157   1.65980218  0.74204416]]


In [7]:
y=(np.random.randn(1,10)>0)
print(y)

[[False False False  True  True False  True  True  True  True]]


In [20]:

def relu(z):
    s=np.maximum(0,z)
    return s


def leakyrelu(z):
    s=np.maximum(0.01*z,z)
    return s


def sigmoid(z):
    s=1/(1+np.exp(-z))
    return s

In [21]:
def layers_size(X,y,node1,node2,node3):
    n_x=X.shape[0]
    n_h1=node1
    n_h2=node2
    n_h3=node3

    n_y=y.shape[0]
    return n_x,n_h1,n_h2,n_h3,n_y

In [22]:

def intz(n_x,n_h1,n_h2,n_h3,n_y):
    w1=np.random.randn(n_h1,n_x)*0.01
    b1=np.zeros((n_h1,1))
    w2=np.random.randn(n_h2,n_h1)*0.01
    b2=np.zeros((n_h2,1))
    w3=np.random.randn(n_h3,n_h2)*0.01
    w4=np.random.randn(n_y,n_h3)*0.01
    b3=np.zeros((n_h3,1))
    b4=np.zeros((n_y,1))
    
    parameters={'w1':w1,'b1':b1,'w2':w2,'b2':b2,'w3':w3,'b3':b3,'w4':w4,'b4':b4}
    return parameters

In [23]:
## forward propogation

def fwd(X,parameters):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]
    w3=parameters["w3"]
    b3=parameters["b3"]
    w4=parameters["w4"]
    b4=parameters["b4"]
    
    
    Z1=np.dot(w1,X)+b1
    A1=relu(Z1)             # np.tanh python fn
    Z2=np.dot(w2,A1)+b2
    A2=relu(Z2)
    Z3=np.dot(w3,A2)+b3
    A3=leakyrelu(Z3)
    Z4=np.dot(w4,A3)+b4
    A4=sigmoid(Z4)
    
    cache={'Z1':Z1,'A1':A1,'Z2':Z2,'A2':A2,'Z3':Z3,'A3':A3,'Z4':Z4,'A4':A4}
    return A4,cache

In [24]:
## cost

def compute_cost(A4,y):
    m=y.shape[1]
    logp=np.multiply(np.log(A4),y)+np.multiply(np.log(1-A4),(1-y))
    cost=-np.sum(logp)/m
    cost=float(np.squeeze(cost))
    return cost

In [25]:
n_x,n_h1,n_h2,n_h3,n_y=layers_size(X,y,4,3,2)

In [26]:
parameters=intz(n_x,n_h1,n_h2,n_h3,n_y)

In [27]:
A4,cache=fwd(X,parameters)

In [28]:
compute_cost(A4,y)

0.6931471803573696

In [29]:
## calculating gradient

def dsigmoid(s):
    das=s*(1-s)
    return das

def dtanh(s):
    dat=(1-s**2)
    return dat

def drelu(s):
    dar=(np.int64(s>0))
    return dar

def dlrelu(s,alpha=0.01):
    dal=np.where(s>0,1,alpha)
    return dal

In [56]:
def BWP(parameters,X,y,cache):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]
    w3=parameters["w3"]
    b3=parameters["b3"]
    w4=parameters["w4"]
    b4=parameters["b4"]
    
    A1=cache['A1']
    A2=cache['A2']
    A3=cache['A3']
    A4=cache['A4']
    
    m=y.shape[1]       ##y.size
    
    dZ4=A4-y
    dW4=np.dot(dZ4,A3.T)/m
    db4=np.sum(dZ4,axis=1,keepdims=True)/m
    dA3=np.dot(w4.T,dZ4)
    dZ3=dA3*dlrelu(A3)
    dW3=np.dot(dZ3,A2.T)/m
    db3=np.sum(dZ3,axis=1,keepdims=True)/m
    dA2=np.dot(w3.T,dZ3)
    dZ2=dA2*drelu(A2)
    dW2=np.dot(dZ2,A1.T)/m
    db2=np.sum(dZ2,axis=1,keepdims=True)/m
    dA1=np.dot(w2.T,dZ2)
    dZ1=dA1*drelu(A1)
    dW1=np.dot(dZ1,X.T)/m
    db1=np.sum(dZ1,axis=1,keepdims=True)/m
    
    
    grades={"dW2":dW2,"db2":db2,"dW1":dW1,"db1":db1,"dW3":dW3,"db3":db3,"dW4":dW4,"db4":db4}

    return grades

In [57]:
##7.Update Grades
def update(parameters,grades,lr=0.01):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]
    w3=parameters["w3"]
    b3=parameters["b3"]
    w4=parameters["w4"]
    b4=parameters["b4"]

    dW1=grades["dW1"]
    db1=grades["db1"]
    dW2=grades["dW2"]
    db2=grades["db2"]
    dW3=grades["dW3"]
    db3=grades["db3"]
    dW4=grades["dW4"]
    db4=grades["db4"]


    w1=w1-lr*dW1
    b1=b1-lr*db1
    w2=w2-lr*dW2
    b2=b2-lr*db2
    w3=w3-lr*dW3
    b3=b3-lr*db3
    w4=w4-lr*dW4
    b4=b4-lr*db4

    parameters={"w1":w1,"b1":b1,"w2":w2,"b2":b2,"w3":w3,"b3":b3,"w4":w4,"b4":b4}

    return parameters

In [58]:

def NN(X,y,itr=10000,print_cost=True,node1=4,node2=3,node3=2):
    np.random.seed(3)
    n_x,n_h1,n_h2,n_h3,n_y=layers_size(X,y,node1,node2,node3)
    
    
    parameters=intz(n_x,n_h1,n_h2,n_h3,n_y)
    
    for i in range(0,itr):
        A4,cache=fwd(X,parameters)
        cost=compute_cost(A4,y)
        grades=BWP(parameters,X,y,cache)
        parameters=update(parameters,grades,lr=0.01)
        
        if print_cost and i%100==0:
            print("cost %i:%f"%(i,cost))
    
    return parameters
    

In [59]:
NN(X,y)

cost 0:0.693147
cost 100:0.685270
cost 200:0.680490
cost 300:0.677584
cost 400:0.675813
cost 500:0.674731
cost 600:0.674068
cost 700:0.673662
cost 800:0.673412
cost 900:0.673259
cost 1000:0.673164
cost 1100:0.673106
cost 1200:0.673070
cost 1300:0.673048
cost 1400:0.673034
cost 1500:0.673025
cost 1600:0.673020
cost 1700:0.673017
cost 1800:0.673015
cost 1900:0.673014
cost 2000:0.673013
cost 2100:0.673012
cost 2200:0.673012
cost 2300:0.673012
cost 2400:0.673012
cost 2500:0.673012
cost 2600:0.673012
cost 2700:0.673012
cost 2800:0.673012
cost 2900:0.673012
cost 3000:0.673012
cost 3100:0.673012
cost 3200:0.673012
cost 3300:0.673012
cost 3400:0.673012
cost 3500:0.673012
cost 3600:0.673012
cost 3700:0.673012
cost 3800:0.673012
cost 3900:0.673012
cost 4000:0.673012
cost 4100:0.673012
cost 4200:0.673012
cost 4300:0.673012
cost 4400:0.673012
cost 4500:0.673012
cost 4600:0.673012
cost 4700:0.673012
cost 4800:0.673012
cost 4900:0.673012
cost 5000:0.673012
cost 5100:0.673012
cost 5200:0.673012
cost 

{'w1': array([[ 0.01788628,  0.0043651 ,  0.00096497, -0.01863493],
        [-0.00277422, -0.00354786, -0.00082766, -0.00626996],
        [-0.00043747, -0.00477187, -0.01313849,  0.00884632],
        [ 0.00881317,  0.01709575,  0.00050035, -0.00404675]]),
 'b1': array([[ 0.00000000e+00],
        [ 2.84256965e-07],
        [-2.17539857e-07],
        [ 3.05554575e-08]]),
 'w2': array([[-0.0054536 , -0.01546485,  0.00982306, -0.01101068],
        [-0.01185047, -0.00205664,  0.01486167,  0.00236723],
        [-0.01023785, -0.00712997,  0.0062525 , -0.00160513]]),
 'b2': array([[-3.36367621e-05],
        [ 2.01635457e-05],
        [ 1.94454447e-06]]),
 'w3': array([[-0.00768845, -0.00230039,  0.00745051],
        [ 0.01976086, -0.01244149, -0.00626434]]),
 'b3': array([[-3.25900093e-05],
        [-9.80856373e-05]]),
 'w4': array([[-0.00803778, -0.024191  ]]),
 'b4': array([[0.40546508]])}